In [231]:
#run this once to set up - then run next cell for each member
# replace encoded1.csv with file paul scraped
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# function for ratings scrape

def user_ratings(username):
    ratings_count = 0
    user_ratings = []
    avail_results = True 
    while avail_results:
        # assemble the url, make the request, isolate table
        url = f"https://www.beeradvocate.com/user/beers/?start={ratings_count}&ba={username}&order=dateD&view=R"
        html = requests.get(url)
        print(html.url)
        soup = BeautifulSoup(html.text, 'html.parser')
         # number of user ratings 
        ######need to dump the ratings from user scrape
        
        results = soup.find_all('table')
        

        #increase ratings count to get next page
        ratings_count += 50
        
        #get correct table rows
        ratings_list = results[0].find_all('tr')[3:] #increase ratings count to get next page
        
        #####need catch for no user ratings
        if len(ratings_list) > 0:
        
            #loop through and sort info
            for tr in ratings_list:
                tds = tr.find_all('td')

                # isolate some info
                user_rating = tds[-2].text
#                 abv = tds[-3].text
                rDev = re.findall(r"([\d.]*\d+)", tds[-1].text)[0]
                # further isolate element in list holding 3 features
                add_beer_info = tds[2].find_all('a')
                
                # extract info
#                 beer_name = add_beer_info[0].text
                beer_link_total = add_beer_info[0]['href']
                base_url_ba = "https://www.beeradvocate.com"
                beer_link = base_url_ba + beer_link_total[0:beer_link_total.find("?")]
#                 beer_style = add_beer_info[2].text
#                 brewery = add_beer_info[1].text

                #dump into dict
                ratings_dict = {
#                     "beer_name": beer_name,
#                     "brewery": brewery,
#                     "beer_style": beer_style,
#                     "beer_style_match": re.sub(r'\W+', '', beer_style.lower()),
                    "beer_link": beer_link,
                    "user_rating": user_rating,
#                     "abv": abv,
                    "rDev": rDev

                }

                # append link to list
                user_ratings.append(ratings_dict)
        else: 
            
            avail_results = False
    user_ratings_df = pd.DataFrame(user_ratings).drop_duplicates()
    
    return user_ratings_df

def match_styles(username):
    #create df of user ratings
    user_ratings_df = user_ratings(username)
    print(len(user_ratings_df))

#   # creates two data frames, one for user_ratings, the other for possible recommendations
    beer_features_df = pd.read_csv("../encoded2.csv")
    user_ratings_df = user_ratings_df.merge(beer_features_df, left_on = 'beer_link', right_on = 'URL', how = "right")
    rec_df = user_ratings_df[user_ratings_df['beer_link'].isnull()]
    user_df = user_ratings_df[user_ratings_df['beer_link'].notnull()]
    user_df['user_rating'] = pd.to_numeric(user_df['user_rating'], errors = "coerce")
    user_df.sort_values('user_rating', ascending = False, inplace=True)
    
    # decision making for 1 and 0
    
    return rec_df, user_df

In [493]:
user = 'LumpkinBrews'
recs, user_df = match_styles(user)

https://www.beeradvocate.com/user/beers/?start=0&ba=LumpkinBrews&order=dateD&view=R
https://www.beeradvocate.com/user/beers/?start=50&ba=LumpkinBrews&order=dateD&view=R
https://www.beeradvocate.com/user/beers/?start=100&ba=LumpkinBrews&order=dateD&view=R
https://www.beeradvocate.com/user/beers/?start=150&ba=LumpkinBrews&order=dateD&view=R
https://www.beeradvocate.com/user/beers/?start=200&ba=LumpkinBrews&order=dateD&view=R
https://www.beeradvocate.com/user/beers/?start=250&ba=LumpkinBrews&order=dateD&view=R
204


/Users/Ronessa/anaconda/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Ronessa/anaconda/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [494]:
len(user_df)

139

In [495]:
user_df.sort_values('user_rating', ascending = False, inplace=True)

In [496]:
key_val = int(len(user_df)*.05)
key_val

6

In [497]:
user_df['yesno'] = ""

In [498]:
user_df['yesno'][0:key_val-1] = 1
user_df['yesno'][key_val-1:] = 0
user_df['yesno'].astype('int', inplace=True)

/Users/Ronessa/anaconda/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/Ronessa/anaconda/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


30     1
43     1
15     1
14     1
34     1
21     0
42     0
8      0
51     0
6      0
47     0
48     0
19     0
28     0
12     0
45     0
10     0
26     0
35     0
39     0
24     0
33     0
57     0
18     0
13     0
69     0
54     0
106    0
114    0
71     0
      ..
74     0
137    0
117    0
131    0
129    0
58     0
62     0
88     0
112    0
75     0
73     0
116    0
67     0
65     0
96     0
91     0
90     0
132    0
92     0
53     0
135    0
95     0
126    0
111    0
66     0
119    0
136    0
130    0
118    0
110    0
Name: yesno, Length: 139, dtype: int64

In [499]:
X = user_df[["Flavor", "ABV","Hoppiness","Maltiness","Fruitiness","Body", "pDev",
          "Sweetness","Sour","spicy","bitter", 'Aroma_None', 'Aroma_caramel',
       'Aroma_floral', 'Aroma_fruit', 'Custom Flavor_None',
       'Custom Flavor_banana', 'Custom Flavor_brown sugar',
       'Custom Flavor_burnt', 'Custom Flavor_caramel',
       'Custom Flavor_chocolate', 'Custom Flavor_earthy',
       'Custom Flavor_herbal', 'Custom Flavor_pumpkin',
       'Custom Flavor_roasted', 'Custom Flavor_smoke', 'Custom Flavor_toasted',
       'Custom Flavor_wheat', 'Color_amber', 'Color_black', 'Color_dark ',
       'Color_gold', 'Color_pale']]


In [500]:
target = user_df['yesno'].astype('int')
target_names = ['good', 'bad']

In [501]:
X.head()

,Flavor,ABV,Hoppiness,Maltiness,Fruitiness,Body,pDev,Sweetness,Sour,spicy,...,Custom Flavor_pumpkin,Custom Flavor_roasted,Custom Flavor_smoke,Custom Flavor_toasted,Custom Flavor_wheat,Color_amber,Color_black,Color_dark,Color_gold,Color_pale
30,2,13.1,2,2,0,3,5.97,2,0,0,...,0,1,0,0,0,0,1,0,0,0
43,3,8.5,3,3,0,2,7.05,0,0,0,...,0,0,0,0,0,1,0,0,0,0
15,2,14.3,2,2,0,3,6.79,2,0,0,...,0,1,0,0,0,0,1,0,0,0
14,2,12.7,2,2,0,3,6.33,2,0,0,...,0,1,0,0,0,0,1,0,0,0
34,2,7.6,1,1,0,2,6.37,0,0,0,...,0,1,0,0,0,0,1,0,0,0


# Decision Tree

In [502]:
feature_names = X.columns
feature_names

Index(['Flavor', 'ABV', 'Hoppiness', 'Maltiness', 'Fruitiness', 'Body', 'pDev',
       'Sweetness', 'Sour', 'spicy', 'bitter', 'Aroma_None', 'Aroma_caramel',
       'Aroma_floral', 'Aroma_fruit', 'Custom Flavor_None',
       'Custom Flavor_banana', 'Custom Flavor_brown sugar',
       'Custom Flavor_burnt', 'Custom Flavor_caramel',
       'Custom Flavor_chocolate', 'Custom Flavor_earthy',
       'Custom Flavor_herbal', 'Custom Flavor_pumpkin',
       'Custom Flavor_roasted', 'Custom Flavor_smoke', 'Custom Flavor_toasted',
       'Custom Flavor_wheat', 'Color_amber', 'Color_black', 'Color_dark ',
       'Color_gold', 'Color_pale'],
      dtype='object')

In [503]:
from sklearn.model_selection import train_test_split
from sklearn import tree

X_train, X_test, y_train, y_test = train_test_split(X, target, random_state = 42)


In [504]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.97142857142857142

In [505]:
predictions = clf.predict(X)
actual = user_df['yesno'].astype('int')
df = pd.DataFrame({"Predict": predictions, "actual": actual})
df['test'] = (df['Predict'] == df['actual'])
df['test'].value_counts()

True     138
False      1
Name: test, dtype: int64

In [506]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.97142857142857142

In [507]:
predictions = rf.predict(X)
actual = user_df['yesno'].astype('int')
df = pd.DataFrame({"Predict": predictions, "actual": actual})
df['test'] = (df['Predict'] == df['actual'])
df['test'].value_counts()

True     138
False      1
Name: test, dtype: int64

In [508]:
recsX = recs[["Flavor", "ABV","Hoppiness","Maltiness","Fruitiness","Body", "pDev",
          "Sweetness","Sour","spicy","bitter", 'Aroma_None', 'Aroma_caramel',
       'Aroma_floral', 'Aroma_fruit', 'Custom Flavor_None',
       'Custom Flavor_banana', 'Custom Flavor_brown sugar',
       'Custom Flavor_burnt', 'Custom Flavor_caramel',
       'Custom Flavor_chocolate', 'Custom Flavor_earthy',
       'Custom Flavor_herbal', 'Custom Flavor_pumpkin',
       'Custom Flavor_roasted', 'Custom Flavor_smoke', 'Custom Flavor_toasted',
       'Custom Flavor_wheat', 'Color_amber', 'Color_black', 'Color_dark ',
       'Color_gold', 'Color_pale']]

In [509]:
recommendations = rf.predict(recsX)

In [510]:
list(recommendations).count(1)

31

In [511]:
len(recommendations)

32641

In [512]:
recs.columns

Index(['beer_link', 'rDev', 'user_rating', 'ABV', 'Availaility', 'Brewer',
       'BrosScore', 'ImageLink', 'Location', 'Name', 'Ranking', 'num_ratings',
       'ReviewCount', 'Score', 'ScoreClass', 'Style', 'URL', 'pDev', 'Flavor',
       'Hoppiness', 'Maltiness', 'Fruitiness', 'Body', 'Sweetness', 'Sour',
       'spicy', 'bitter', 'Aroma_None', 'Aroma_caramel', 'Aroma_floral',
       'Aroma_fruit', 'Custom Flavor_None', 'Custom Flavor_banana',
       'Custom Flavor_brown sugar', 'Custom Flavor_burnt',
       'Custom Flavor_caramel', 'Custom Flavor_chocolate',
       'Custom Flavor_earthy', 'Custom Flavor_herbal', 'Custom Flavor_pumpkin',
       'Custom Flavor_roasted', 'Custom Flavor_smoke', 'Custom Flavor_toasted',
       'Custom Flavor_wheat', 'Color_amber', 'Color_black', 'Color_dark ',
       'Color_gold', 'Color_pale'],
      dtype='object')

In [513]:
len(recs)

32641

In [514]:
recsCut1 = recs[recs['Score'] > 4.0]

In [515]:
recsCut = recsCut1[["Flavor", "ABV","Hoppiness","Maltiness","Fruitiness","Body", "pDev",
          "Sweetness","Sour","spicy","bitter", 'Aroma_None', 'Aroma_caramel',
       'Aroma_floral', 'Aroma_fruit', 'Custom Flavor_None',
       'Custom Flavor_banana', 'Custom Flavor_brown sugar',
       'Custom Flavor_burnt', 'Custom Flavor_caramel',
       'Custom Flavor_chocolate', 'Custom Flavor_earthy',
       'Custom Flavor_herbal', 'Custom Flavor_pumpkin',
       'Custom Flavor_roasted', 'Custom Flavor_smoke', 'Custom Flavor_toasted',
       'Custom Flavor_wheat', 'Color_amber', 'Color_black', 'Color_dark ',
       'Color_gold', 'Color_pale']]

In [516]:
len(recsCut)

9071

In [517]:
recommendations = rf.predict(recsCut)

In [518]:
list(recommendations).count(1)

28

In [519]:
recsCut1['recs'] = recommendations

/Users/Ronessa/anaconda/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [520]:
recsCut1[recsCut1['recs'] == 1].groupby('Style')['recs'].count()

Style
American Double / Imperial IPA       3
American Double / Imperial Stout    25
Name: recs, dtype: int64